In [ ]:
# ─── ① imports ─────────────────────────────────────────────
import geopandas as gpd
from pathlib import Path
import shutil, os

In [ ]:
# ─── ② 参数配置 ────────────────────────────────────────────
input_root  = Path(r'D:/gis/to_transform')     # 待转换文件根目录
output_root = Path(r'D:/gis/transformed')      # 输出根目录
output_root.mkdir(parents=True, exist_ok=True)

target_crs = 'EPSG:4326'          # 目标：WGS-84
assumed_source_crs = None         # 如果数据本身没 CRS，请在这里填如 'EPSG:4547' 等

In [ ]:
# ─── ③ 核心函数 ────────────────────────────────────────────
def transform_vector(in_path: Path):
    """Read vector file, re-project to `target_crs`, write to mirror location."""
    try:
        gdf = gpd.read_file(in_path)
    except Exception as e:
        print(f'[ERR] {in_path.name}: {e}')
        return
    
    if gdf.crs is None:
        if not assumed_source_crs:
            print(f'[SKIP] {in_path.name}: no CRS & no assumed_source_crs.')
            return
        gdf = gdf.set_crs(assumed_source_crs)
    
    if str(gdf.crs).upper() == str(target_crs).upper():
        print(f'[=] {in_path.name}: already in {target_crs}')
        return
    
    gdf_out = gdf.to_crs(target_crs)
    rel     = in_path.relative_to(input_root)
    out_path = (output_root / rel).with_suffix('.shp')  # 始终写 Shapefile
    out_path.parent.mkdir(parents=True, exist_ok=True)
    gdf_out.to_file(out_path)
    print(f'[OK] {in_path} → {out_path}')

In [ ]:
# ─── ④ 批处理执行 ──────────────────────────────────────────
vector_exts = {'.shp', '.geojson', '.json', '.gpkg'}
files = [p for p in input_root.rglob('*') if p.suffix.lower() in vector_exts]

print(f'Total vector files: {len(files)}')
for vec in files:
    transform_vector(vec)